In [ ]:
# AI and ML - group project

# Importing libraries
from sklearn.metrics import silhouette_score
from pandas.plotting import parallel_coordinates
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Importing dataset
df = pd.read_csv(
    r'C:/Users/silva/Documents/Faculdade/Intercâmbio/AI and Machine Learning/Group project/customer_segmentation.csv')
print(df)

# Fixing the date
datelist = ['order_purchase_timestamp', 'order_approved_at', 'order_approved_at', 'order_delivered_carrier_date',
            'order_delivered_customer_date', 'order_estimated_delivery_date', 'shipping_limit_date']

# Converting the list of date variables to a pandas datetime object
for c in datelist:
    df[c] = pd.to_datetime(df[c])

# Separating the date and time in the order column they might be interesting variables for the analysis
df['order_date'] = [d.date() for d in df['order_purchase_timestamp']]
df['order_time'] = [d.time() for d in df['order_purchase_timestamp']]

# Taking a look at the data structure
df.head()
df.info()
df.describe()
df.shape

# Data preparation

# Removing duplicates and resetting the index of the dataframe
df.duplicated().sum()
df.drop_duplicates(keep='first', inplace=True)
df.reset_index(drop=True, inplace=True)

# Checking for missing values
df.isnull().sum()

# Checking for outliers
plt.figure()
df.reset_index().plot(kind='scatter', x='index',
                      y='payment_installments', c='brown')
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='payment_value', c='gray')

# Checking the mean and standar deviation of the payment value (to check how far is the outlier observed in the scatterplot)
mean_pv = df["payment_value"].mean()  # mean of 195.2
std_pv = df["payment_value"].std()  # std of 295.5

# Seeing how many outliers there are, the output prints 5 payments higher than 4000
# Value of 4000 decided by looking at the scatterplot
outlier1 = df[df['payment_value'] > 4000]
print('\nOutlier dataframe:\n', outlier1)

# Deleting 5 outliers out of 13718 other orders and resetting the index again
df = df.drop(df[df.payment_value > 4000].index)
df = df.reset_index(drop=True)

# Continuing to look at other variables by plotting scatterplots
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='payment_value',
                      c='gray')  # re-plotting after removing outliers
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='price', c='orange')
plt.figure()
df.reset_index().plot(kind='scatter', x='index', y='freight_value', c='purple')
plt.figure()

# Creating a correlation matrix heatmap to look at the relation between variables
sns.heatmap(df.corr(), annot=True)
plt.figure()

# Counting customers per city and per state and creating dataframes with the purpose of plotting a graph
customerstate = df.groupby('customer_state').count()[
    'customer_id'].reset_index()
customercity = df.groupby('customer_city').count()['customer_id'].reset_index()

# Graph of numer of customers in each state
plt.figure()
sns.barplot(data=customerstate.sort_values('customer_id',
            ascending=False), x='customer_state', y='customer_id')
plt.title('Number of Customers per State', fontweight='bold')
plt.xlabel('State')
plt.ylabel('Number of Customers')

# Graph of top 10 cities with most customers
plt.figure()
sns.barplot(data=customercity.sort_values('customer_id', ascending=False).nlargest(
    10, 'customer_id'), x='customer_id', y='customer_city')
plt.title('Cities with more Customers', fontweight='bold')
plt.xlabel('City')
plt.ylabel('Number of Customers')

# Grouping similar categories of product category names in case we use this variable in future analysis (encoding becomes easier, there would be less dummies)
df['product_category_name_english'] = df['product_category_name_english'].replace(['art', 'arts_and_craftmanship', 'sports_leisure',
                                                                                   'garden_tools', 'flowers', 'music', 'musical_instruments',
                                                                                   'books_general_interest', 'books_imported', 'books_technical'], 'hobbies')
df['product_category_name_english'] = df['product_category_name_english'].replace(['air_conditioning', 'bed_bath_table', 'furniture_bedroom',
                                                                                   'furniture_decor', 'furniture_living_room', 'home_appliances',
                                                                                   'home_appliances_2', 'home_comfort_2', 'home_confort',
                                                                                   'home_construction', 'housewares', 'kitchen_dining_laundry_garden_furniture',
                                                                                   'small_appliances', 'small_appliances_home_oven_and_coffee', 'office_furniture',
                                                                                   'signaling_and_security', 'stationery', 'luggage_accessories'], 'home_products')
df['product_category_name_english'] = df['product_category_name_english'].replace(
    ['drinks', 'food', 'food_drink'], 'food_drink')
df['product_category_name_english'] = df['product_category_name_english'].replace(['construction_tools_construction', 'construction_tools_lights',
                                                                                   'construction_tools_safety', 'costruction_tools_garden',
                                                                                   'costruction_tools_tools'], 'construction_tools')
df['product_category_name_english'] = df['product_category_name_english'].replace(['audio', 'auto', 'cds_dvds_musicals', 'cine_photo', 'computers',
                                                                                   'computers_accessories', 'consoles_games', 'dvds_blu_ray', 'electronics',
                                                                                   'fixed_telephony', 'telephony', 'tablets_printing_image'], 'electronic_gadgets')
df['product_category_name_english'] = df['product_category_name_english'].replace(['fashion_bags_accessories', 'fashion_childrens_clothes', 'fashion_male_clothing',
                                                                                   'fashion_shoes', 'fashion_underwear_beach', 'health_beauty', 'perfumery'], 'fashion_beauty')
df['product_category_name_english'] = df['product_category_name_english'].replace(['baby', 'diapers_and_hygiene', 'toys', 'party_supplies', 'pet_shop',
                                                                                   'christmas_supplies', 'cool_stuff', 'watches_gifts'], 'family_festivities')
df['product_category_name_english'] = df['product_category_name_english'].replace(
    ['industry_commerce_and_business', 'market_place'], 'sellers')


# Counting the product categories to check which one is most and less ordered
best_seller_p = df['product_category_name_english'].value_counts(
).reset_index().nlargest(5, 'product_category_name_english')
worst_seller_p = df['product_category_name_english'].value_counts(
).reset_index().nsmallest(5, 'product_category_name_english')

# Graphs aesthetic
plt.figure(figsize=(15, 12))
green_color = sns.color_palette()[3]
red_color = sns.color_palette()[2]

# Graph of top 10 most ordered products
plt.subplot(211)
sns.barplot(data=best_seller_p, x='product_category_name_english',
            y='index', color=green_color)
plt.title('Top 5 Product Categories Ordered', fontweight='bold')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category')

# Graph of top 10 less ordered products
plt.subplot(212)
sns.barplot(data=worst_seller_p, x='product_category_name_english',
            y='index', color=red_color)
plt.title('Lowest 5 Product Categories Ordered', fontweight='bold')
plt.xlabel('Number of Orders')
plt.ylabel('Product Category')

# Counting different payment types to plot which ones are more used
payments_types = df['payment_type'].value_counts().reset_index()

plt.figure(figsize=(25, 7))

# Graph of number of orders per payment type
plt.subplot(121)
sns.barplot(data=payments_types, x='index', y='payment_type')
plt.title('Orders by Payment type', fontweight='bold')
plt.xlabel('Payment Type')
plt.ylabel('Number of Orders')

# Graph of number of orders per number of payment installments
plt.subplot(122)
sns.barplot(data=df['payment_installments'].value_counts(
).reset_index(), x='index', y='payment_installments')
plt.title('Count of Orders With Number of Payment Installments', fontweight='bold')
plt.xlabel('Number of Payment Installments')
plt.ylabel('Count of Orders')

# Encoding variables
df = pd.get_dummies(df, prefix=['customer_state: '], columns=[
                    'customer_state'], drop_first=False)
df = pd.get_dummies(df, prefix=['payment_type: '], columns=[
                    'payment_type'], drop_first=False)
df = pd.get_dummies(df, prefix=['seller_state: '], columns=[
                    'seller_state'], drop_first=False)
df = pd.get_dummies(df, prefix=['product_category_name: '], columns=[
                    'product_category_name_english'], drop_first=False)

# Calculating the RFM

# Recency = time since a customer's last purchase
# Calculating each purchasing time stamp minus the most recent purchase timestamp (max)
df['recency'] = max(df['order_purchase_timestamp']) - \
    df['order_purchase_timestamp']
# Getting the minimum recency value for each customer (customer with multiple purchases ended up with multiple recencies, therefore calculating the most recent purchase)
df['recency'] = df.groupby(['customer_unique_id'], as_index=False)[
    'recency'].transform('min')
# Keeping only the days in the column, leaving out the time
df['recency'] = df['recency'].dt.days

# Frequency = how many times has a customer made a purchase
# Counting purchases per customer unique id and adding values to a new column in the dataset
df['frequency'] = df.groupby(['customer_unique_id'], as_index=False)[
    'order_id'].transform('count')

# Monetary = total amount a customer has spend purchasing products
# Calculating it by summing all the payment values a customer has spent
df['monetary'] = df.groupby(['customer_unique_id'], as_index=False)[
    'payment_value'].transform('sum')

# Creating an empty column to calculate recency score
df['recency_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "how many months could we consider the customer inactive?"
print('recency mean: ', df['recency'].mean())
print('recency std: ', df['recency'].std())
print('recency max: ', df['recency'].max())
print('recency min: ', df['recency'].min())

for i in df.index:
    if df['recency'][i] <= 30:
        df['recency_score'][i] = 5
    elif (df['recency'][i] > 30) and (df['recency'][i] <= 60):
        df['recency_score'][i] = 4
    elif (df['recency'][i] > 60) and (df['recency'][i] <= 120):
        df['recency_score'][i] = 3
    elif (df['recency'][i] > 120) and (df['recency'][i] <= 180):
        df['recency_score'][i] = 2
    elif (df['recency'][i] > 180):
        df['recency_score'][i] = 1

# Creating an empty column to calculate frequency score
df['frequency_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "most customers purchased only one time, and max purchases were 13, how can we score from 0 to above"
print('frequency mean: ', df['frequency'].mean())
print('frequency std: ', df['frequency'].std())
print('frequency max: ', df['frequency'].max())
print('frequency min: ', df['frequency'].min())

for i in df.index:
    if df['frequency'][i] >= 10:
        df['frequency_score'][i] = 5
    elif (df['frequency'][i] >= 6) and (df['frequency'][i] < 10):
        df['frequency_score'][i] = 4
    elif (df['frequency'][i] >= 4) and (df['frequency'][i] < 6):
        df['frequency_score'][i] = 3
    elif (df['frequency'][i] >= 2) and (df['frequency'][i] < 4):
        df['frequency_score'][i] = 2
    elif (df['frequency'][i] == 1):
        df['frequency_score'][i] = 1

# Creating an empty column to calculate monetary score
df['monetary_score'] = ''

# For function that inputs the score in the empty column based on the conditions set
# The numbers were chosen based of the recency mean, std, max and min + our reasoning considered aspects like "the mean of the payment was 395.1, and the std is 1090.5, how can we score the customers based on spending"
print('monetary mean: ', df['monetary'].mean())
print('monetary std: ', df['monetary'].std())
print('monetary max: ', df['monetary'].max())
print('monetary min: ', df['monetary'].min())

for i in df.index:
    if df['monetary'][i] > 500:
        df['monetary_score'][i] = 5
    elif (df['monetary'][i] > 250) and (df['monetary'][i] <= 500):
        df['monetary_score'][i] = 4
    elif (df['monetary'][i] > 150) and (df['monetary'][i] <= 250):
        df['monetary_score'][i] = 3
    elif (df['monetary'][i] > 100) and (df['monetary'][i] <= 150):
        df['monetary_score'][i] = 2
    elif (df['monetary'][i] <= 100):
        df['monetary_score'][i] = 1

# Uniting the scores in the same columns to find segments such as 555 (5 score for all rfm)
df['rfm_segment'] = df['recency_score'].astype(
    str) + df['frequency_score'].astype(str) + df['monetary_score'].astype(str)

# Calculating total score (sum of all val)

# Turning all numbers from last created columns to numeric variables to use in future analysis
df['rfm_segment'] = pd.to_numeric(df['rfm_segment'])
df['recency_score'] = pd.to_numeric(df['recency_score'])
df['frequency_score'] = pd.to_numeric(df['frequency_score'])
df['monetary_score'] = pd.to_numeric(df['monetary_score'])

# Creating a dataframe with the rfm variables that are going to be used in the clustering algorithms
rfm = df[['recency', 'frequency', 'monetary']]

# Plotting a few exploratory graphs for rfm

# Customers who order 1 time and 2 times are predominant. There fewer customers are the ones that have ordered more than 5 times.
plt.figure(figsize=(8, 5))
sns.distplot(df['frequency'], bins=8, kde=False, rug=True)
plt.figure()

# The ammount customers spend is frequently less than 300$.
plt.figure(figsize=(8, 5))
sns.distplot(df['monetary'], kde=False, rug=True)
plt.xlim(0, 2000)

# Customers who had come within the last 2 months are the majority and there are some customers that have not ordered for more than a year.
plt.figure(figsize=(8, 5))
sns.distplot(df['recency'], bins=8, kde=False, rug=True)

# Standardizing the features
scaler = StandardScaler()
rfm_std = scaler.fit_transform(rfm)

# Segmenting the custumer in Gold, Silver and Bronze according to their scores in frequency, recency and monetary value
# and keeping in mind that we assinged to recency a lower weight since in our opinion is a less impotant factor to determine future customer behavior.
df['customer_category'] = ((df['monetary_score']*2) +
                           (df['frequency_score']*2) + (df['recency_score']*1)) / 5

for i in df.index:

    if (df['customer_category'][i] >= 4) and (df['customer_category'][i] <= 5):
        df['customer_category'][i] = 'Gold'
    elif (df['customer_category'][i] > 2) and (df['customer_category'][i] < 4):
        df['customer_category'][i] = 'Silver'
    elif (df['customer_category'][i] >= 1) and (df['customer_category'][i] <= 2):
        df['customer_category'][i] = 'Bronze'

def generate_legend_labels(chart_data, chart_labels):
    total_values = sum(chart_data.values)

    for key, value in chart_data.items():
        value_percentage = value / total_values * 100
        rounded_value_percentage = round(value_percentage, 1)
        chart_labels.append(key + ' - ' + str(rounded_value_percentage) + '%')

# Plotting the percentege of bronze, silver and gold customers
colors = ['#8B7355', '#C0C0C0', '#EEC900']
customer_category = df.groupby(['customer_category']).count().sort_values(
    by=['customer_id'], ascending=False)['customer_id']

customer_category_labels = []
customer_category_sizes = customer_category.values
generate_legend_labels(customer_category, customer_category_labels)

# Generates the pie chart
customer_category_chart, customer_category_ax = plt.subplots()
customer_category_ax.pie(customer_category_sizes,
                         shadow=True, startangle=90, colors=colors)
customer_category_ax.axis('equal')

plt.legend(labels=customer_category_labels, loc='upper left',
           bbox_to_anchor=(-0.25, 1.), fontsize=9)
customer_category_ax.set_title(
    'Percentage of each customer segment', fontweight='bold')
plt.show()

# Clustering algorithms

# K means clustering

# Importing libraries for K-means algorithm and performance measurments

# Using elbow method to find out optimal k value with WCSS score
# Creating empty list to insert values
WCSS = []

# Using a for function to fill the list with the WCSS scores
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(rfm_std)
    WCSS.append(kmeans.inertia_)

# Plotting the Elbow method graph
plt.figure()
plt.plot(range(1, 11), WCSS, marker='o', label='line with marker')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters=3, init='k-means++', random_state=42)
# Use fit_predict to cluster the dataset
km_clusters = kmeans.fit_predict(rfm_std)

rfm_std_cl = pd.DataFrame(rfm_std, columns=rfm.columns, index=rfm.index)
rfm_std_cl['cluster'] = km_clusters

# Visualising the clusters (3 clusters according to the Elbow method)
plt.scatter(rfm_std[km_clusters == 0, 0], rfm_std[km_clusters ==
            0, 1], s=30, c='orange', label='Cluster 1')
plt.scatter(rfm_std[km_clusters == 1, 0], rfm_std[km_clusters ==
            1, 1], s=30, c='brown', label='Cluster 2')
plt.scatter(rfm_std[km_clusters == 2, 0], rfm_std[km_clusters ==
            2, 1], s=30, c='gray', label='Cluster 3')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[
            :, 1], s=150, c='black', label='Centroids')
plt.title('Clusters of customers')
plt.legend()
plt.show()

# Calculating cluster validation metrics (silhouette score, calinski harabasz score, davies bouldin score)
score_kemans_s = silhouette_score(rfm_std, kmeans.labels_, metric='euclidean')
score_kemans_c = calinski_harabasz_score(rfm_std, kmeans.labels_)
score_kemans_d = davies_bouldin_score(rfm_std, km_clusters)
print('Silhouette Score: %.3f' % score_kemans_s)
print('Calinski Harabasz Score: %.3f' % score_kemans_c)
print('Davies Bouldin Score: %.3f' % score_kemans_d)

# Hierarchical clustering

# Plotting the dendrogram to find the optimal number of clusters
plt.figure()
dendrogram = sch.dendrogram(sch.linkage(rfm_std, method='ward'))
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.xticks([])
plt.show()

# Fitting Hierarchical Clustering to the dataset
hc = AgglomerativeClustering(
    n_clusters=4, affinity='euclidean', linkage='ward')
hc_clusters = hc.fit_predict(rfm_std)

# Visualising the clusters (4 clusters according to the Dendogram)
plt.figure()
plt.scatter(rfm_std[hc_clusters == 0, 0], rfm_std[hc_clusters == 0, 1], s = 30, c = 'orange', label = 'Cluster 1')
plt.scatter(rfm_std[hc_clusters == 1, 0], rfm_std[hc_clusters == 1, 1], s = 30, c = 'brown', label = 'Cluster 2')
plt.scatter(rfm_std[hc_clusters == 2, 0], rfm_std[hc_clusters == 2, 1], s = 30, c = 'gray', label = 'Cluster 3')
plt.scatter(rfm_std[hc_clusters == 3, 0], rfm_std[hc_clusters == 3, 1], s = 30, c = 'yellow', label = 'Cluster 4')
plt.title('Clusters of customers')
plt.legend()
plt.show()

# Calculating cluster validation metrics
score_AGclustering_s = silhouette_score(
    rfm_std, hc.labels_, metric='euclidean')
score_AGclustering_c = calinski_harabasz_score(rfm_std, hc.labels_,)
score_AGclustering_d = davies_bouldin_score(rfm_std, hc_clusters)
print('Silhouette Score: %.3f' % score_AGclustering_s)
print('Calinski Harabasz Score: %.3f' % score_AGclustering_c)
print('Davies Bouldin Score: %.3f' % score_AGclustering_d)

# DBSCAN clustering

# Since the DBSCAN code runs only with dataframes, first, we transformed the standardized array to a DataFrame
rfm_std_df = pd.DataFrame(rfm_std, columns=rfm.columns)

# Tuning the hyperparameters

# Finding the 'n'
min_sample = 2*len(rfm_std_df.columns)
print(min_sample)  # equal to 6

# Finding the epsilon
# n = 7 because the first column will be of zeros
nbrs = NearestNeighbors(n_neighbors=7).fit(rfm_std_df)
# finding the k neighbors to each point
neigh_dist, neigh_ind = nbrs.kneighbors(rfm_std_df)
sort_neigh_dist = np.sort(neigh_dist, axis=0)  # sorting in ascending order

# Plotting the graph to visualize which is the optimal epsilon value
k_dist = sort_neigh_dist[:, 6]
plt.plot(k_dist)
# 0.4 chosen as it crosses the middle of the curve in the graph = optimal epsilon
plt.axhline(y=0.4, linewidth=1, linestyle='dashed', color='k')
plt.ylabel("k-NN distance")
plt.xlabel("Observations")
plt.show()

# Fitting the model
dbscan_model = DBSCAN(eps=0.4, min_samples=6).fit(rfm_std_df)
labels = dbscan_model.labels_
rfm_std_df["LABEL"] = labels
rfm_std_df.head(10)

# Plotting the graph to find the clusters
g = sns.PairGrid(rfm_std_df, hue='LABEL', palette="Paired")
g = g.map(sns.scatterplot)
print(g)

# Fitting model and predicting clusters
dbscan_clusters = dbscan_model.fit_predict(rfm_std_df)

# Calculating cluster validation metrics
score_dbsacn_s = silhouette_score(
    rfm_std_df, dbscan_clusters, metric='euclidean')
score_dbsacn_c = calinski_harabasz_score(rfm_std_df, dbscan_clusters)
score_dbsacn_d = davies_bouldin_score(rfm_std_df, dbscan_clusters)
print('Silhouette Score: %.3f' % score_dbsacn_s)
print('Calinski Harabasz Score: %.3f' % score_dbsacn_c)
print('Davies Bouldin Score: %.3f' % score_dbsacn_d)

# Considering the validation metrics, K-means was our best model

# Interpreting the clusters in K-means
print('Size of each cluster: ')
print(rfm_std_cl['cluster'].value_counts())

# Separating clusters in different dataframes
cluster0 = pd.DataFrame()
cluster1 = pd.DataFrame()
cluster2 = pd.DataFrame()

for i in rfm_std_cl.index:
    if rfm_std_cl['cluster'][i] == 0:
        cluster0 = cluster0.append(rfm_std_cl.iloc[i])
    elif rfm_std_cl['cluster'][i] == 1:
        cluster1 = cluster1.append(rfm_std_cl.iloc[i])
    elif rfm_std_cl['cluster'][i] == 2:
        cluster2 = cluster2.append(rfm_std_cl.iloc[i])

# Plotting the clusters in relation to other variables

# 'New' customers = high recency, low frequency or monetary
plt.figure()
plt.subplot(311)
parallel_coordinates(cluster0, 'cluster', color=('#556270'))
plt.ylim(0, 15)
# 'Lost' customers = moderate frequency and monetary, low recency
plt.subplot(312)
parallel_coordinates(cluster1, 'cluster', color=('#4ECDC4'))
plt.ylim(0, 15)
# 'Loyal' customers = high monetary value, above average frequency, moderate recency
plt.subplot(313)
parallel_coordinates(cluster2, 'cluster', color=('#C7F464'))
plt.ylim(0, 15)
plt.tight_layout()
































